# MEcon Teaching Plan → Select Courses → iOS Calendar (.ics)

This notebook reads your MEcon teaching plan DOCX, lists all detected `CourseCode+Class` combos (e.g., `ECON6001D`), lets you edit a list of desired codes/classes, and generates an iOS-compatible `.ics` file.

In [ ]:
# Parameters
from pathlib import Path

# Use repo-relative paths

DOCX_PATH = "/Users/Documents/teaching plan 2025-26 full courses"  # absolute path/
TIMEZONE = "Asia/Hong_Kong"
OUTPUT_ICS = "/Users/Documents/teaching_plan_selected_from_notebook.ics"

# Edit this list to choose desired CourseCode+Class combos
# Examples: ["ECON6001D", "ECON6093"]
DESIRED_CODES = ["ECON6001D", "ECON6093"]


In [ ]:
# Install runtime deps if needed
import sys, subprocess

def ensure(pkg):
    try:
        __import__(pkg)
    except Exception:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])

for mod in ['python_docx', 'icalendar']:
    name = 'python-docx' if mod == 'python_docx' else mod
    try:
        __import__(mod if mod != 'python_docx' else 'docx')
    except Exception:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', name])
print('OK')


In [ ]:
import datetime as dt
import re
from pathlib import Path
from typing import Dict, List, Optional, Sequence, Tuple

from docx import Document
from icalendar import Calendar, Event


def normalize(text: str) -> str:
    return re.sub(r"\s+", " ", (text or "").strip())


def detect_default_year(texts: Sequence[str], fallback: int = 2025) -> int:
    joined = "\n".join(texts)
    m = re.search(r"(20\d{2})\s*[-/]\s*(\d{2})", joined)
    if m:
        return int(m.group(1))
    m2 = re.search(r"\b(20\d{2})\b", joined)
    return int(m2.group(1)) if m2 else fallback


def parse_time_range(text: str):
    t = text.strip()
    m = re.search(r"(\d{1,2}):(\d{2})\s*(am|pm)?\s*(?:-|–|—|~|to)\s*(\d{1,2}):(\d{2})\s*(am|pm)?", t, re.I)
    if not m:
        return None
    h1, m1, ap1, h2, m2, ap2 = m.groups()
    h1 = int(h1); m1 = int(m1); h2 = int(h2); m2 = int(m2)
    if ap1:
        ap1 = ap1.lower();
        if ap1 == 'pm' and h1 != 12: h1 += 12
        if ap1 == 'am' and h1 == 12: h1 = 0
    if ap2:
        ap2 = ap2.lower();
        if ap2 == 'pm' and h2 != 12: h2 += 12
        if ap2 == 'am' and h2 == 12: h2 = 0
    return dt.time(h1, m1), dt.time(h2, m2)


def parse_date_tokens(text: str, default_year: int) -> List[dt.date]:
    dates: List[dt.date] = []
    # YYYY-M-D
    for m in re.finditer(r"\b(20\d{2})[./-](\d{1,2})[./-](\d{1,2})\b", text):
        y, mo, d = map(int, m.groups()); dates.append(dt.date(y, mo, d))
    # D/M/YYYY
    for m in re.finditer(r"\b(\d{1,2})[./-](\d{1,2})[./-](20\d{2})\b", text):
        d, mo, y = map(int, m.groups()); dates.append(dt.date(y, mo, d))
    # D/M (assume HK format D/M with default year)
    for m in re.finditer(r"\b(\d{1,2})[./-](\d{1,2})\b", text):
        d, mo = map(int, m.groups())
        if 1 <= d <= 31 and 1 <= mo <= 12:
            dates.append(dt.date(default_year, mo, d))
    # Month name (Sep 10, 2025 or Sep 10) and Day-MonthName (2-Sep)
    month_map = {m.lower(): i for i, m in enumerate(["", "January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]) if m}
    abbr = {k[:3].lower(): v for k, v in month_map.items()}
    for m in re.finditer(r"\b([A-Za-z]{3,9})\.?\s*(\d{1,2})(?:,\s*(\d{4}))?\b", text):
        mon = m.group(1).lower(); day = int(m.group(2)); year = int(m.group(3)) if m.group(3) else default_year
        mo = month_map.get(mon) or abbr.get(mon)
        if mo: dates.append(dt.date(year, mo, day))
    for m in re.finditer(r"\b(\d{1,2})\s*[-/]\s*([A-Za-z]{3,9})\s*(?:,?\s*(\d{4}))?\b", text):
        day = int(m.group(1)); mon = m.group(2).lower(); year = int(m.group(3)) if m.group(3) else default_year
        mo = month_map.get(mon) or abbr.get(mon)
        if mo: dates.append(dt.date(year, mo, day))
    return sorted({d for d in dates})


def list_code_class_combos(docx_path: Path) -> List[str]:
    doc = Document(str(docx_path))
    combos = set()
    for table in doc.tables:
        if not table.rows: continue
        headers = [normalize(c.text).lower() for c in table.rows[0].cells]
        try:
            code_idx = next((i for i,h in enumerate(headers) if 'course code' in h or 'course code & title' in h or h=='code'), None)
            class_idx = next((i for i,h in enumerate(headers) if h.strip()=='class'), None)
        except Exception:
            code_idx = class_idx = None
        if code_idx is None: continue
        for r in table.rows[1:]:
            cells = r.cells
            code_text = normalize(cells[code_idx].text)
            m = re.search(r"\b([A-Z]{4}\d{4})\b", code_text)
            base = m.group(1) if m else ''
            clazz = normalize(cells[class_idx].text) if class_idx is not None else ''
            if base:
                combos.add((base + (clazz or '')).upper())
    return sorted(combos)


def parse_meetings(docx_path: Path, desired_codes: Sequence[str]) -> List[Tuple[str, str, dt.date, dt.time, dt.time, str]]:
    doc = Document(str(docx_path))
    default_year = detect_default_year([p.text for p in doc.paragraphs])
    desired_pairs = []
    for c in desired_codes:
        m = re.match(r"^([A-Z]{4}\d{4})([A-Z]?)$", c.strip().upper())
        if m:
            desired_pairs.append((m.group(1), m.group(2)))
    out = []
    for table in doc.tables:
        if not table.rows: continue
        headers = [normalize(c.text).lower() for c in table.rows[0].cells]
        idx_code = next((i for i,h in enumerate(headers) if 'course code' in h or 'course code & title' in h or h=='code'), None)
        idx_class = next((i for i,h in enumerate(headers) if h.strip()=='class'), None)
        idx_title = next((i for i,h in enumerate(headers) if 'course title' in h or h=='title' or 'name' in h or 'course code & title' in h), idx_code)
        idx_time = next((i for i,h in enumerate(headers) if 'time' in h), None)
        idx_dates = next((i for i,h in enumerate(headers) if 'date' in h or 'schedule' in h), None)
        idx_loc = next((i for i,h in enumerate(headers) if 'venue' in h or 'location' in h or 'room' in h), None)
        if idx_code is None: continue
        for r in table.rows[1:]:
            cells = r.cells
            code_text = normalize(cells[idx_code].text)
            m = re.search(r"\b([A-Z]{4}\d{4})\b", code_text)
            base = (m.group(1) if m else '').upper()
            clazz = normalize(cells[idx_class].text).upper() if idx_class is not None else ''
            if not base: continue
            matched = None
            for (req_base, req_cls) in desired_pairs:
                if base != req_base: continue
                if req_cls:
                    if clazz == req_cls or clazz == '':
                        matched = (req_base, req_cls)
                        break
                else:
                    matched = (req_base, clazz)
                    break
            if not matched: continue
            title = normalize(cells[idx_title].text)
            tstr = normalize(cells[idx_time].text) if idx_time is not None else ''
            dstr = normalize(cells[idx_dates].text) if idx_dates is not None else ''
            loc = normalize(cells[idx_loc].text) if idx_loc is not None else ''
            tr = parse_time_range(tstr or dstr) or parse_time_range(" ".join(normalize(c.text) for c in cells))
            if not tr: continue
            start_t, end_t = tr
            dates = parse_date_tokens(dstr or tstr, default_year)
            if not dates:
                dates = parse_date_tokens(" ".join(normalize(c.text) for c in cells), default_year)
            for d in dates:
                out.append((base + (matched[1] or clazz), title or base, d, start_t, end_t, loc))
    return out


def build_calendar(meetings, tz_name: str) -> Calendar:
    try:
        from zoneinfo import ZoneInfo
        tz = ZoneInfo(tz_name)
    except Exception:
        import pytz
        tz = pytz.timezone(tz_name)
    cal = Calendar()
    cal.add('prodid', '-//Teaching Plan Notebook//iOS//')
    cal.add('version', '2.0')
    for codecls, title, d, start_t, end_t, loc in meetings:
        ev = Event()
        ev.add('summary', f"{codecls} - {title}")
        if loc: ev.add('location', loc)
        ev.add('dtstart', dt.datetime.combine(d, start_t).replace(tzinfo=tz))
        ev.add('dtend', dt.datetime.combine(d, end_t).replace(tzinfo=tz))
        cal.add_component(ev)
    return cal


In [ ]:
# List all detected CourseCode+Class combos
combos = list_code_class_combos(Path(DOCX_PATH))
print(f"Detected {len(combos)} combos (sample):")
print("\n".join(combos[:50]))


In [ ]:
# Parse desired courses and write ICS
meetings = parse_meetings(Path(DOCX_PATH), DESIRED_CODES)
print(f"Parsed {len(meetings)} meetings")
cal = build_calendar(meetings, TIMEZONE)
Path(OUTPUT_ICS).write_bytes(cal.to_ical())
print(f"Wrote {OUTPUT_ICS}")
